In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import matplotlib.font_manager as fm
from sklearn.datasets import load_boston

# # GPU 설정 : 런타임 > 런타임 유형 변경 > GPU
# # GPU 사용 가능한지 확인
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# 데이터 프레임화
bst = load_boston()

bst_x=pd.DataFrame(data=bst.data,columns=bst.feature_names)
bst_y=pd.DataFrame(bst.target,columns=["MEDV"])
bst_all=pd.merge(bst_x,bst_y,left_index=True,right_index=True)
bst_all

## 레버리지와 아웃라이어

개별적인 데이터 표본 하나하나가 회귀분석 결과에 미치는 영향력은 레버리지 분석이나 아웃라이어 분석을 통해 알 수 있다

레버리지는 독립변수 평균으로부터 데이터가 얼마나 멀리 떨어져 있는지를 나타내는 것으로 독립변수에서 특이값 찾는 것이다.

이상치는 종속변수에서 특이값을 찾는 것이다.


#### 레버리지
실제 종속변수값 y가 예측치(predicted target) y^에 미치는 영향을 나타낸 값이다. self-influence, self-sensitivity 라고도 한다.

가중치 벡터의 결과값을 예측식에 대입하여 y 와 y^의 관계를 다음과 같다는 것을 보였었다.

y^=Hy
이 행렬 H를 영향도 행렬(influence matrix) 또는 hat 행렬(hat matrix)이라고 한다고 하였다.

행렬 H의 i번째 행, j번째 열 성분을 hij라고 하면 실제 결과값 yi과 예측값 y^i은 다음과 같은 관계를 가진다.

[레버리지 특성]
1. 1보다 같거나 작은 양수 혹은 0이다.
2. 레버리지의 합은 모형에 사용된 모수의 갯수 K와 같다. 모수에는 상수항도 포함되므로 상수항이 있는 1차원 모형에서는 K=2가 된다.
tr(H)=∑iNhii=K

이 두가지 성질로부터 레버리지 값은 N개의 데이터에 대한 레버리지값은 양수이고 그 합이 K가 된다는 것을 알 수 있다. 즉 K라고 하는 값을 N개의 변수가 나누어 가지는 것과 같다. 현실적으로 데이터의 갯수 N는 모수의 갯수 K보다 훨씬 많기 때문에 위에서 말한 특성을 적용하면 모든 레버리지 값이 동시에 1이 되는 것은 불가능하다.

위 식을 이용하면 레버리지의 평균값도 구할 수 있다.

hii≈KN

보통 이 평균값의 2 ~ 4배보다 레버리지 값이 크면 레버리지가 크다고 이야기한다

In [ ]:
#statsmodels를 이용한 레버리지 계산

plt.scatter(bst_all["RM"],bst_all["MEDV"])
plt.show()

In [ ]:
# 선형회귀
Y=bst_all["MEDV"]
X=bst_all[["RM","DIS"]]
model=sm.OLS(Y,X)
result=model.fit()

print(result.summary())

In [ ]:
## 영향도 정보
infl=result.get_influence()
hat=infl.hat_matrix_diag

plt.figure(figsize=(18,2))
plt.stem(hat)
plt.axhline(hat.mean(),c="r",ls="--")
# plt.axhline? : Add a horizontal line across the axis.
# 대부분의 데이터는 레버리지 값이 0.0 근처의 낮은 값을 가진다.
plt.show()

# 대부분의 데이터는 레버리지 값이 0.00197 근처의 낮은 값을 가진다.
hat.mean()

In [ ]:
ax = plt.subplot()
plt.scatter(X.iloc[:,-1], Y)
sm.graphics.abline_plot(model_results=result, ax=ax)

idx = hat > 0.05
plt.scatter(X[idx], Y[idx], s=300, c="r", alpha=0.5)
plt.title("회귀분석 결과와 레버리지 포인트")
plt.show()


In [ ]:
# 레버리지의 영향¶
'''
레버리지가 큰 데이터가 모형에 주는 영향을 보기 위해 이 데이터가 포함된 경우의 모형과 포함되지 않은 경우의 모형을 아래에 비교하였다. 
레버리지가 큰 데이터는 포함되거나 포함되지 않는가에 따라 모형에 주는 영향이 큰 것을 알 수 있다.
'''
X2=bst_all[["PTRATIO","NOX"]]
Y2=bst_all[["MEDV"]]
model2 = sm.OLS(X2,Y2)
result2 = model2.fit()


ax = plt.subplot()
plt.scatter(bst_all["RM"], bst_all["MEDV"])
sm.graphics.abline_plot(model_results=result,c="r",linestyle="--", ax=ax)
sm.graphics.abline_plot(model_results=result2,c="g", alpha=0.7, ax=ax)


# plt.plot(X0[-1], y[-1], marker='x', c="m", ms=20, mew=5)
# plt.legend([u"레버리지가 큰 데이터를 포함한 경우", u"레버리지가 큰 데이터를 포함하지 않은 경우"],
#            loc="upper left")
# plt.title("레버리지가 높은 데이터가 회귀분석에 미치는 영향")
# plt.show()

## 아웃라이어¶
모형에서 설명하고 있는 데이터와 동떨어진 값을 가지는 데이터, 즉 잔차가 큰 데이터를 **아웃라이어(outlier)**라고 한다. 그런데 잔차의 크기는 독립 변수의 영향을 받으므로 아웃라이어를 찾으러면 이 영향을 제거한 표준화된 잔차를 계산해야 한다.

잔차를 레버리지와 잔차의 표준 편차로 나누어 동일한 표준 편차를 가지도록 스케일링한 것을 표준화 잔차(standardized residual 또는 normalized residual 또는 studentized residual)라고 한다.


In [ ]:
## 잔차 계산
plt.stem(result.resid)
plt.show()

In [ ]:
## 표준화 잔차
plt.figure(figsize=(20, 4))
plt.stem(result.resid_pearson)
plt.axhline(5,c="b",ls="--")
plt.axhline(-5,c="b",ls="--")
plt.title("각 데이터의 표준화 잔차")
plt.show()

### Cook’s Distance¶
회귀 분석에는 잔차의 크기가 큰 데이터가 아웃라이어가 되는데 이 중에서도 주로 관심을 가지는 것은 레버리지와 잔차의 크기가 모두 큰 데이터들이다. 잔차와 레버리지를 동시에 보기위한 기준으로는 Cook’s Distance가 있다. 다음과 같이 정의되는 값으로 레버리지가 커지거나 잔차의 크기가 커지면 Cook’s Distance 값이 커진다.

Di=(ri^2/RSS)*[hii(1−hii)2]

Fox’ Outlier Recommendation 은 Cook’s Distance가 다음과 같은 기준값보다 클 때 아웃라이어로 판단하자는 것이다.

Di>4N−K−1

모든 데이터의 레버리지와 잔차를 동시에 보려면 plot_leverage_resid2 명령을 사용한다. 
이 명령은 x축으로 표준화 잔차의 제곱을 표시하고 y축으로 레버리지값을 표시한다. 
데이터 아이디가 표시된 데이터들이 레버리지가 큰 아웃라이어이다.

In [ ]:
plt.figure(figsize=(20, 20))
sm.graphics.plot_leverage_resid2(result)
plt.show()

In [ ]:
sm.graphics.influence_plot(result, plot_alpha=0.3)
plt.show()

In [ ]:
from statsmodels.graphics import utils
# Fox’Outlier Recommendation 은 Cook’s Distance가 Di>4N−K−1보다 클 때 아웃라이어로 판단
cooks_d2, pvals = influence.cooks_distance
K = influence.k_vars
fox_cr = 4 / (len(y) - K - 1)
idx = np.where(cooks_d2 > fox_cr)[0]

ax = plt.subplot()

X2=np.array(X2)
Y2=np.array(Y2)

plt.scatter(X2,Y2)
plt.scatter(X2[idx], Y2[idx], s=300, c="r", alpha=0.5)
utils.annotate_axes(range(len(idx)), idx,
                    list(zip(X2[idx], Y2[idx])), [(-20, 15)] * len(idx), size="small", ax=ax)
plt.title("Fox Recommendaion으로 선택한 아웃라이어")
plt.show()

#### 보스턴예제

In [ ]:
model_bst = sm.OLS(bst_y, bst_x)
result_bst =model_bst.fit()
pred = result_bst.predict(bst_x)

influence_bst = result_bst.get_influence()
cooks_d2, pvals = influence_bst.cooks_distance
K = influence.k_vars
fox_cr = 4 / (len(y) - K - 1)
idx = np.where(cooks_d2 > fox_cr)[0]

# MEDV = 50 제거
idx = np.hstack([idx, np.where(bst.target == 50)[0]])

ax = plt.subplot()
plt.scatter(bst_y, pred)
plt.scatter(bst_y.MEDV[idx], pred[idx], s=300, c="r", alpha=0.5)
utils.annotate_axes(range(len(idx)), idx,
                    list(zip(bst_y.MEDV[idx], pred[idx])), [(-20, 15)] * len(idx), size="small", ax=ax)
plt.title("보스턴 집값 데이터에서 아웃라이어")
plt.show()

In [ ]:
# 다음은 이렇게 아웃라이어를 제외한 후에 다시 회귀분석을 한 결과이다.
idx2 = list(set(range(len(bst_x))).difference(idx))

bst_x = bst_x.iloc[idx2, :].reset_index(drop=True)
bst_y = bst_y.iloc[idx2, :].reset_index(drop=True)

model_boston2 = sm.OLS(bst_y, bst_x)
result_boston2 = model_boston2.fit()
print(result_boston2.summary())